# Task 5

### Dependencies

- urllib3
    - we will be using urllib for our networking 
- bs4
    - we will be using beautiful soup 4 to be doing the parsing of the html

In [1]:
import urllib3
import string
import json
import time
from bs4 import BeautifulSoup
from bs4.dammit import EncodingDetector


We can start by defining our group number which will be used to choose which parts of the site we will be scraping

In [2]:
group_nr = 0
parser = 'html.parser'

Now we want to define some way that we will be retrieving the pages

In [3]:

http = urllib3.PoolManager()

def soup_from_url(url):
    resp = http.request('GET', url).data.decode('utf-8')
    return BeautifulSoup(resp, parser)


To scrape the page, we choose to use the `&pagefrom` parameter in the page for the category, and iterate it for each letter in the group that was given in the assignment text. We then iterate all links checking if the title of the link corresponds to the letter were trying to retrieve. Then we retrieve the page in the link for the article extracting information to it and storing it in a `dict` tree structure.

In [4]:
scrape_tree = {}
count_pages = 0


for l in string.ascii_uppercase[group_nr%23:group_nr%23+10]:
    host = "https://en.wikinews.org"
    page = "/w/index.php?title=Category:Politics_and_conflicts&pagefrom={0}#mw-pages".format(l)
    soup = soup_from_url(host + page)
    for link in soup.find_all('a', href=True):
        title = link.parent.text
        if title.lower().startswith(l.lower()):
            a_page = link['href']
            a_soup = soup_from_url(host + a_page)
            scrape_tree[page] = {}
            count_pages += 1
            try:
                scrape_tree[page][a_page] = {
                    "content": a_soup.find("div", {"id": "mw-content-text"}).text,
                    "date": a_soup.find("span", {"id": "publishDate"})['title'],
                    "title": title
                }
            except TypeError as e:
                continue

MaxRetryError: HTTPSConnectionPool(host='en.wikinews.orghttps', port=443): Max retries exceeded with url: //creativecommons.org/licenses/by/2.5/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x1147348e0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

At the end we write the retireved data to a json file.

In [ ]:
with open('scraped.json', 'w') as outfile:
    json.dump(scrape_tree, outfile)
